# Apriori Algorithm Implementation Assignment

### Objective:
You will implement the **Apriori algorithm** from scratch (i.e., without using any libraries like `mlxtend`) to find frequent itemsets and generate association rules.

### Dataset:
Use the [Online Retail Dataset](https://www.kaggle.com/datasets/vijayuv/onlineretail) from Kaggle. You can filter it for a specific country (e.g., `United Kingdom`) and time range to reduce size if needed.

---

## Step 1: Data Preprocessing

- Load the dataset
- Remove rows with missing values
- Filter out rows where `Quantity <= 0`
- Convert Data into Basket Format

👉 **Implement code below**

In [1]:
# Load the dataset
# Preprocess as per the instructions above | We have already done in TASK 2
import pandas as pd
data = pd.read_csv('OnlineRetail.csv')
data = data.dropna()
data = data[ data['Quantity'] >= 0]
data = data[data['Country'] == 'United Kingdom']

# Your Code Here
# for Basket
basket = data.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
basket = basket.map(lambda x: 1 if x > 0 else 0)
basket

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
len(basket)

16649

## Step 2: Implement Apriori Algorithm
Step-by-Step Procedure:
1. Generate Frequent 1-Itemsets
Count the frequency (support) of each individual item in the dataset.
Keep only those with support ≥ min_support.
→ Result is L1 (frequent 1-itemsets)
2. Iterative Candidate Generation (k = 2 to n)
While L(k-1) is not empty:
a. Candidate Generation

Generate candidate itemsets Ck of size k from L(k-1) using the Apriori property:
Any (k-itemset) is only frequent if all of its (k−1)-subsets are frequent.
b. Prune Candidates
Eliminate candidates that have any (k−1)-subset not in L(k-1).
c. Count Support
For each transaction, count how many times each candidate in Ck appears.
d. Generate Frequent Itemsets
Form Lk by keeping candidates from Ck that meet the min_support.
Repeat until Lk becomes empty.
Implement the following functions:
1. `get_frequent_itemsets(transactions, min_support)` - Returns frequent itemsets and their support
2. `generate_candidates(prev_frequent_itemsets, k)` - Generates candidate itemsets of length `k`
3. `calculate_support(transactions, candidates)` - Calculates the support count for each candidate

**Write reusable functions** for each part of the algorithm.

In [72]:
# Implement apriori functions below

def get_frequent_itemsets(transactions, min_support):
    # Your code here
    pass

def generate_candidates(prev_frequent_itemsets, k):
    # Your code here
    pass

def calculate_support(transactions, candidates):
    # Your code here
    pass

In [34]:
def get_frequent_itemsets(transactions, min_support):
    #This list will contain all the frequent itemsets
    frequentItemSets = []

    #This list will contain all the non-frequent itemsets which will be used for pruning
    nonFrequentItemSets = [[]]

    # This format is used to create a new entry in the Frequent Item df 
    ItemSetFormat = {
        'ItemSet' : [],
        'SupportCount': []
    }
    candidateSet = transactions.sum().reset_index()
    candidateSet.columns = ['ItemSet', 'SupportCount']
    candidateSet['ItemSet'] = candidateSet['ItemSet'].map( lambda x : {x} )
    frequentItemSets.append(pd.DataFrame(ItemSetFormat))
    currentItemSets = frequentItemSets[0]
    for i in range(0, len(candidateSet)):
        itemSet = candidateSet.iloc[i]
        if itemSet.SupportCount >= min_support:
            currentItemSets.loc[len(currentItemSets)] = itemSet
        else:
            # print('nonFreqItemSet:', itemSet['ItemSet'])
            nonFrequentItemSets[0].append(itemSet['ItemSet'])
    
    flag = True
    itemSetSize = 2
    while(flag):
        frequentItemSets.append(pd.DataFrame(ItemSetFormat))
        itemSet = generate_candidates(frequentItemSets[itemSetSize-2]['ItemSet'], itemSetSize)
        currentItemSets = frequentItemSets[itemSetSize-1]

        for s in itemSet:
            isFrequent = True
            for nonFrequentItemSet in nonFrequentItemSets[-1]:
                if nonFrequentItemSet.issubset(s):
                    isFrequent = False
                    nonFrequentItemSets[-1].append(s)
                    break

            if isFrequent:
                supportCount = calculate_support(transactions, s)
                if supportCount >= min_support:
                    currentItemSets.loc[len(currentItemSets)] = [s, supportCount]
        itemSetSize += 1
        if len(currentItemSets) == 1:
            # print('Frequent Patterns: ', frequentItemSets)
            return currentItemSets
        elif len(currentItemSets) < 1:
            return frequentItemSets[-2]
    
def generate_candidates(prev_frequent_itemsets, k):
    candidateSets = []
    for s1 in prev_frequent_itemsets:
        for s2 in prev_frequent_itemsets:
            s = s1.union(s2)
            if len(s) == k and s not in candidateSets:
                candidateSets.append(s)
    return candidateSets

def calculate_support(transactions, candidates):
    candidates = list(candidates)
    column = transactions[candidates[0]]
    for i in range(1, len(candidates)):
        column = column & transactions[candidates[i]]
    return column.sum()


In [36]:
freqPattern = get_frequent_itemsets(basket, 400)
print('\n\n')
# print('Item Set: ',freqPattern)
freqPattern

,ItemSet,SupportCount
0,"{ALARM CLOCK BAKELIKE RED , ALARM CLOCK BAKELI...",454
1,"{GARDENERS KNEELING PAD KEEP CALM , GARDENERS ...",458
2,"{PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY...",404
3,"{ROSES REGENCY TEACUP AND SAUCER , GREEN REGEN...",476
4,"{JUMBO BAG PINK POLKADOT, JUMBO BAG RED RETROS...",506
5,"{JUMBO STORAGE BAG SUKI, JUMBO BAG RED RETROSPOT}",408
6,"{LUNCH BAG BLACK SKULL., LUNCH BAG CARS BLUE}",402
7,"{LUNCH BAG BLACK SKULL., LUNCH BAG PINK POLKA...",442
8,"{LUNCH BAG BLACK SKULL., LUNCH BAG RED RETROS...",484
9,"{LUNCH BAG RED RETROSPOT, LUNCH BAG CARS BLUE}",413


In [37]:
freqPattern.iloc[0].ItemSet

{'ALARM CLOCK BAKELIKE GREEN', 'ALARM CLOCK BAKELIKE RED '}

## Step 3: Generate Association Rules

- Use frequent itemsets to generate association rules
- For each rule `A => B`, calculate:
  - **Support**
  - **Confidence**
- Only return rules that meet a minimum confidence threshold (e.g., 0.5)

👉 **Implement rule generation function below**

In [ ]:
# Function to generate rules from frequent itemsets

def generate_rules(frequent_itemsets, min_confidence):
    pass

## Step 4: Output and Visualize

- Print top 10 frequent itemsets
- Print top 10 association rules (by confidence or lift)

👉 **Output results below**

In [ ]:
# Output the final results
# Optional: Add visualizations

# Your Code Here